In [1]:
pip install transformers faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Cargar el modelo de embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Cargar información desde un archivo externo
with open("Unal.txt", "r", encoding="utf-8") as f:
    university_info = f.read()

# Dividir el texto en oraciones
sentences = [sentence.strip() for sentence in university_info.split('.') if sentence.strip()]

# Convertir las oraciones en vectores
sentence_embeddings = model.encode(sentences)

# Crear un índice FAISS
dimension = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(sentence_embeddings))

In [ ]:
from transformers import pipeline

# Cargar un modelo generativo (GPT-2 en este caso)
generator = pipeline('text-generation', model='gpt2')

def rag_chat(question):
    # Convertir la pregunta en un vector
    question_embedding = model.encode([question])

    # Buscar las oraciones más relevantes
    distances, indices = index.search(question_embedding, k=1)
    relevant_sentence = sentences[indices[0][0]]

    # Formatear el prompt para el modelo generativo
    prompt = (
        f"Basado en la siguiente información, responde la pregunta de manera clara y concisa.\n"
        f"Información: {relevant_sentence}\n"
        f"Pregunta: {question}\n"
        f"Respuesta:"
    )

    # Generar una respuesta basada en la información recuperada
    response = generator(
        prompt,
        max_new_tokens=20,  # Limitar la longitud de la respuesta
        num_return_sequences=1
    )

    return response[0]['generated_text']

# Ejemplos de preguntas
questions = [
    "¿Cuándo comienza el proceso de admisión para el 2024?",
    "¿Dónde está ubicada la sede principal de la Universidad Nacional?",
    "¿Qué programas ofrece la Universidad Nacional?",
    "¿La Universidad Nacional tiene programas de posgrado?",
    "¿En qué ciudad está la sede principal de la Universidad Nacional?"
]

# Probar el sistema con las preguntas
for question in questions:
    response = rag_chat(question)
    print(f"Pregunta: {question}")
    print(f"Respuesta: {response}\n")

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Pregunta: ¿Cuándo comienza el proceso de admisión para el 2024?
Respuesta: Basado en la siguiente información, responde la pregunta de manera clara y concisa.
Información: Pregunta: ¿Cuándo comienza el proceso de admisión para el 2024?
Respuesta: El proceso de admisión para el 2024 comienza en enero
Pregunta: ¿Cuándo comienza el proceso de admisión para el 2024?
Respuesta: El proceso de admisión para el 2024 comienza en enero comi-



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Pregunta: ¿Dónde está ubicada la sede principal de la Universidad Nacional?
Respuesta: Basado en la siguiente información, responde la pregunta de manera clara y concisa.
Información: Pregunta: ¿Dónde está ubicada la sede principal de la Universidad Nacional?
Respuesta: La sede principal está ubicada en Bogotá, Colombia
Pregunta: ¿Dónde está ubicada la sede principal de la Universidad Nacional?
Respuesta: La sede principal está ubicada en Bogotá, Colombia
Respuesta:



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Pregunta: ¿Qué programas ofrece la Universidad Nacional?
Respuesta: Basado en la siguiente información, responde la pregunta de manera clara y concisa.
Información: Pregunta: ¿Qué programas ofrece la Universidad Nacional?
Respuesta: Ofrece programas de pregrado y posgrado en diversas áreas
Pregunta: ¿Qué programas ofrece la Universidad Nacional?
Respuesta: Forávía que ha reconno de todo hacer es lo que de estos en



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Pregunta: ¿La Universidad Nacional tiene programas de posgrado?
Respuesta: Basado en la siguiente información, responde la pregunta de manera clara y concisa.
Información: Pregunta: ¿La Universidad Nacional tiene programas de posgrado?
Respuesta: Sí, ofrece programas de posgrado
Pregunta: ¿La Universidad Nacional tiene programas de posgrado?
Respuesta: Sí, ofrece programas de posgrado á rejemplo una

Pregunta: ¿En qué ciudad está la sede principal de la Universidad Nacional?
Respuesta: Basado en la siguiente información, responde la pregunta de manera clara y concisa.
Información: Pregunta: ¿En qué ciudad está la sede principal de la Universidad Nacional?
Respuesta: La sede principal está en Bogotá, Colombia
Pregunta: ¿En qué ciudad está la sede principal de la Universidad Nacional?
Respuesta: ¿En qué ciudad está la sede principal de la Université de

